<a href="https://colab.research.google.com/github/mcstllns/UNIR2024/blob/main/Unir_ejemplo02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio 02. Clasificación binaria con datos de infidelidad

Se analizan unos datos obtenido de Kaggle.com sobre una encuesta de infidelidad: [enlace](https://www.kaggle.com/datasets/utkarshx27/fairs-extramarital-affairs-data)


Las variables son:

**Criterio**: affairs.

**Predictoras**: gender, age,	yearsmarried,	children,	religiousness,	education,	occupation,	rating.


La variable criterio es de tipo dicotómico 0 = no, 1 = sí por lo que el análisis convencional nos lleva a un modelo binomial. La precisión (*accuracy*) en la clasificación obtenida con este modelo es de **0.72**

In [ ]:
# Paquetes y librerías que vamos a necesitar

import tensorflow as tf

print(tf.__version__)

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input

from keras.utils import plot_model
from keras.utils import set_random_seed

import time

In [ ]:
# Definimos algunas funciones que vamos a necesitar

# hace un plot de la historia de ajuste
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.plot(hist['epoch'], hist['accuracy'],'r--',
           label='Training Error')
  # plt.plot(hist['epoch'], hist['val_mse'],'b',
  #          label = 'Validation Error')
  plt.ylim([0,1.0])
  plt.axhline(y=0.72, color='b', linestyle='-')
  plt.legend()
  plt.show() # un €

# normaliza un conjunto de datos
def norm(x, st):
    return((x - st['mean'])/st['std'])



In [ ]:
# Cargamos los datos desde mi Github (mcstllns/UNIR2024)

url = 'https://raw.githubusercontent.com/mcstllns/UNIR2024/main/data-affairs.csv'
data  = pd.read_csv(url)
print(data.keys())
data.head()


In [ ]:
# El fichero tiene datos perdidos y hay que eliminar las filas donde estan
data = data.dropna()

In [ ]:
# Creamos los 4 conjuntos de datos x e y, train y test

x_train = data.sample(frac=0.8, random_state=0)
x_test = data.drop(x_train.index)

y_train = x_train.pop('affairs')
y_test = x_test.pop('affairs')

In [ ]:
x_train.head()

In [ ]:
y_train.head()

In [ ]:
# Vamos a normalizar los conjuntos de datos

# calculamos los estadisticos descriptivos para x_train
x_train_stats = x_train.describe().transpose()
x_train_stats.head()

# calculamos los estadisticos descriptivos para x_test
x_test_stats = x_test.describe().transpose()
x_test_stats.head()

x_train_norm = norm(x_train, x_train_stats)
x_train_norm.head()

x_test_norm = norm(x_test, x_test_stats)
x_test_norm.head()

In [ ]:
# comprobamos que la normalizacion ha funcionado

x_train_norm.describe().transpose()

In [ ]:
# Modelo --------------------------


# Configuramos la topología

model = Sequential()
model.add(Input(shape=(len(x_train_norm.keys()),)))
model.add(Dense(4, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

model.summary()

In [ ]:
# Compilamos el modelo, definimos los hiperparámetros
#  - Función de pérdida: binary_crossentropy
#  - Algoritmo de aprendizaje: Adam con lr = 0.001
#  - Métricas: accuracy

set_random_seed(1)

model.compile(loss='binary_crossentropy',
              optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001),
              metrics = ['accuracy'])

In [ ]:
history = model.fit(x_train_norm, y_train,
                    epochs = 50,
                    verbose = 1
                    ) # para evitar que se llene toda la pantalla

In [ ]:
# hacemos un plot para ver si aprende adecuadamente
plot_history(history)

In [ ]:
# Evaluamos el modelo con el conjunto de entrenamiento

train_loss, train_acc = model.evaluate(x_train_norm, y_train)  # se evalua la precision de la red

In [ ]:
# Evaluamos el modelo con el conjunto de test

test_loss, test_acc = model.evaluate(x_test_norm, y_test)  # se evalua la precision de la red

In [ ]:
# Podemos meter nuestros propios valores (normalizados) para ver la prediccion

mip = pd.DataFrame(np.array([[0, 20, 1, 0, 0, 20, 1, 5]]), columns=x_train_norm.keys())
mip_norm = norm(mip, x_test_stats)
print(mip_norm)

model.predict(mip_norm)


Esta estrategia de ir variando los valores de las variables predictoras y ver cómo afecta a la predicción se puede sistematizar y urilizarse para valorar la relevancia de las variables.

Por ejemplo, en nuestro caso parece que el nivel educativo es altamente predictivo (**más nivel de estudios más probabilidad de ser infiel**)